In [ ]:
#input must be options chain day1 - end

In [44]:
!pip install python-dotenv
!pip install duckdb
!pip install tqdm
!pip install minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.9 MB/s eta 0:00:00


In [1]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from IPython.display import clear_output
import joblib
import random
import hashlib
from src.wgangp.utils import Scaler

In [60]:
CSV_PATH     = './data/OptionsEOD.csv/'
#create @ part 1
PARQUET_PATH = './data/OptionsEOD.parquet'
#create @ part 2
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = [ 'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
#create @ part 3
OTHER_COL = ['OPTIONS_ID','QUOTE_UNIXTIME', 'EXPIRE_UNIX']

QUOTE_COL = ['C_BID',	'C_ASK',  'P_BID',	'P_ASK']
VEGA_COL =  ["C_VEGA","P_VEGA"] 
VOLUME_COL =  ["TOTAL_VOLUME","C_VOLUME","P_VOLUME"] 

SCALER_QUOTE_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in QUOTE_COL]
SCALER_VEGA_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VEGA_COL]
SCALER_VOLUME_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VOLUME_COL]
SCALER_OTHER_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ]


UNIQUE_KEYS = ['SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainRenforcement/'
START = True#True#'2010-09'#True
BUCKET = 'tensorflow'

max_option_len = 16
#==== Create options_qoute
options_qoute = {}

In [ ]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

In [ ]:
==================================Create OptionsEOD.parquet part 1========================================

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    #scaler = MinMaxScaler()
    scaler = MinMaxScaler()
    
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)
                EOD_CSV.dropna(subset=[SCALER_COL])
            
                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [ ]:
==================================Create OptTrainData STG part 2========================================

In [3]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()  # By default, it loads the .env file from the current directory

# Example: Access environment variables
import os

In [4]:
os.getenv('MINIO_HOST')

'host.containers.internal:9000'

In [5]:
import duckdb
con = duckdb.connect(database=':memory:')
con.execute(f"""
CREATE SECRET secretMinioProd (
    TYPE S3,
    KEY_ID '{os.getenv('MINIO_USER')}',
    SECRET '{os.getenv('MINIO_SECRET')}',
    ENDPOINT '{os.getenv('MINIO_HOST')}',
    USE_SSL 'false',
    URL_STYLE 'path'
);
""")

In [67]:
##### Part II 
from tqdm import tqdm

#START = '2012-02'
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
# ###===== load Scaler
# SCALER = Scaler(
#     [v for v in SCALER_COL if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ] + ['QUOTE','VEGA','VOLUME']
#     , create=True
#     , path="./data/scaler/")

if START == True:
    pass
###===== local functions
def strikeZero(df,v,qv,num_rm):   
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == qv['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    if num_rm != 0 :
        df.loc[ (df['QUOTE_DATE'] == qv['QUOTE_DATE']) 
                & (df['SYMBOL'] == v['SYMBOL']) 
                & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
                & (  (df['STRIKE'].isin(filtered_arr[:num_rm])  )
                   | (df['STRIKE'].isin(filtered_arr[-num_rm:])   )
                  ) 
            , SCALER_COL    
        ] = 0
    
def hash_str(S):
    return int(hashlib.md5(S.encode('utf-8')).hexdigest(),16)
    
#========================================= main model =========================================
keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()

#==== Get PartitionDate
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
#PartitionDate = ['2011-12','2022-05'] # debug
debug = None
for i,partdate in  enumerate(tqdm(PartitionDate[:1]) ) :  
    START = True if START == partdate else START
    if START == True :
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        df['PartitionDate'] = df['PartitionDate'].astype('object') 
        #add col options_id
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
        df[df == 0] = 0
        df = df.dropna(subset=[c for c in SCALER_COL if c not in ('ID','QUOTE_DATE', 'EXPIRE_DATE', 'TOTAL_VOLUME') ])
        ####################################################
        keys_item = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()[:]
        
        #loop each UNIQUE_KEYS keys 
        for j,v in keys_item.iterrows():
            msg = {}
            
            msg['symbo'] = v['SYMBOL']
            msg['exdate'] = v['EXPIRE_DATE']
            
            df_Item=df[(df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            keys_quote = df_Item[['QUOTE_DATE']].sort_values(by=['QUOTE_DATE']).drop_duplicates()[:]
            DATA = pd.DataFrame([],columns=OTHER_COL+SCALER_COL)
            for k,qv in keys_quote.iterrows():
                msg['qd_ate'] = qv['QUOTE_DATE']
                df_filter =df_Item[(df_Item['QUOTE_DATE'] == qv['QUOTE_DATE']) ]
                #df_filter['OPTIONS_ID'] = df_filter['OPTIONS_ID'].astype('int') 
                # df_filter['EXPIRE_DATE'] = df['EXPIRE_DATE'].astype('S10') 
                # df_filter['OPTIONS_ID'] = df['OPTIONS_ID'].astype('S10') 
                msg['start_rows'] = len(df_filter) 

                #break loops
                if len(df_filter) < 5 : 
                    print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (less rows)""")
                    break 

                qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
                #add new qoute
                if qoute not in [*options_qoute.keys()]:
                    options_qoute[qoute] = {}
                    options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                    options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                    options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                    #check diff UNDERLYING_LAST
                    if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                        print('[ERROR] : set UNDERLYING_LAST ',qoute )
                #rm index max : max_option_len
                #may be initstrike out of scope
                rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
                df_filter = df_filter.drop(rm_strike_index)
    
                #break loops
                if len(df_filter) < 5 :  
                    print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (rm_strike_index)""")
                    break
                    
                # Generate zero strike 
                # Generate a random float between 0.01 and 1
                
                # dte = df_filter[ df_filter['DTE'] != 0]['DTE'].mean()
                # df_filter['DTE'] = dte
                
                strikeZero(df_filter,v,qv,3 )
                df_filter['OPTIONS_ID'] = int( f'{i}{j}{k}' )
                # #++++++++++++++++++++++++++++++++++++++++++++++
                # print( "A")
                # print( len(df_filter) )
                # print( nom_rm_rows )
                # print( df_filter['STRIKE_DISTANCE'] )
                # break
                # #++++++++++++++++++++++++++++++++++++++++++++++
                
                #====filed rows
                z = 0
                while len(df_filter) != max_option_len :
                    z += 1
                    if z > 50: raise "error"
                    Even= -1
                    if (len(df_filter) % 2) == 0:
                        Even = 0
                    if len(df_filter) < max_option_len :
                        zero_row = pd.DataFrame([[0]*len(df_filter.columns)], columns=df_filter.columns)
                        if Even==0 :
                            df_filter = pd.concat([df_filter, zero_row], ignore_index=True)
                        else:
                            df_filter = pd.concat([zero_row, df_filter], ignore_index=True)
                    elif len(df_filter) > max_option_len :
                         df_filter = df_filter.drop(df_filter.index[Even])
                
                #add cal columns.      
                msg['end_rows']= len(df_filter)
                if len(df_filter) == max_option_len and np.sum( df_filter[OTHER_COL+SCALER_COL].isna().values ) == 0 :
                    #DATA = np.vstack((DATA ,[df_filter[OTHER_COL+SCALER_COL]]))   
                    df_filter_reindexed = df_filter.reindex(columns=OTHER_COL+SCALER_COL)
                    df_filter_reindexed['QUOTE_INDEX'] = range(1, len(df_filter_reindexed) + 1)
                    if len(DATA) :
                        DATA = pd.concat([DATA, df_filter_reindexed], ignore_index=True)
                    else:
                        DATA = df_filter_reindexed

                #if len(DATA) :
                    # #========================== partial_fit ================================#
                    # for col_i in SCALER_OTHER_COL_INDEX:
                    #     SCALER()[SCALER_COL[col_i]].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,[col_i]] )
                    # SCALER()['QUOTE'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_QUOTE_COL_INDEX].reshape(-1, 1) )
                    # SCALER()['VEGA'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VEGA_COL_INDEX].reshape(-1, 1) )
                    # SCALER()['VOLUME'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VOLUME_COL_INDEX].reshape(-1, 1) )
                    # SCALER.save()
                    
            # #=== clear expire options_qoute
            # for qi in list(options_qoute.keys()):
            #     if options_qoute[qi]['exp'] < quote:
            #         options_qoute.pop(qi)
            
            #========================== SAVE DATA End for k============================#
            # #=== save H5
            # if not os.path.exists(H5_PATH):
            #     os.makedirs(H5_PATH)
            # with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
            #     dset = f.create_dataset(f'{i}{j}{k}', data=DATA, chunks=True , compression='gzip')
            # #=== save parquet minio
            con.register('df_table', DATA)
            parquet_file_path = f"s3://tensorflow/renforcement/rawdata/{partdate}/{v['SYMBOL']}_{str(v['EXPIRE_DATE'])[:10]}.parquet"
            con.execute(f"COPY df_table TO '{parquet_file_path}' (FORMAT PARQUET);")
            
            # #=== save parquet
            # if os.path.exists(PARQUET_STG_PATH):
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
            # else:
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )   
            debug = f"[Last Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%       "

100%|██████████| 1/1 [00:14<00:00, 14.86s/it]


In [ ]:
================= duckdb loaddata ================

In [57]:
from minio import Minio
# Initialize MinIO client
minio_client = Minio(
    os.getenv('MINIO_HOST'),  # Change to your MinIO server address
    access_key=os.getenv('MINIO_USER'),
    secret_key=os.getenv('MINIO_SECRET'),
    secure=False  # Use False if not using HTTPS
)

In [68]:
object_list = []
for _dir in minio_client.list_objects(BUCKET, prefix='renforcement/rawdata/'):
    for file in minio_client.list_objects(BUCKET, prefix=_dir.object_name):
        object_list.append(file.object_name)

'renforcement/rawdata/2012-01/QQQ_2012-01-06.parquet'

In [71]:
rawdata=con.execute(f"""
SELECT * FROM read_parquet('s3://tensorflow/{object_list[0]}', filename = true);
""").fetchdf()

In [72]:
rawdata

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME,QUOTE_INDEX,filename
0,0,1325624400,1325883600,0.00,0.0,0.0,0.00,0.0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00,0.0,1,s3://tensorflow/renforcement/rawdata/2012-01/Q...
1,0,1325624400,1325883600,0.00,0.0,0.0,0.00,0.0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00,0.0,2,s3://tensorflow/renforcement/rawdata/2012-01/Q...
2,0,1325624400,1325883600,0.00,0.0,0.0,0.00,0.0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00,0.0,3,s3://tensorflow/renforcement/rawdata/2012-01/Q...
3,0,1325624400,1325883600,56.90,52.0,4.9,4.90,3.0,0.00000,0.00159,4.84,4.92,0.0,0.0,0.01,100.0,4,s3://tensorflow/renforcement/rawdata/2012-01/Q...
4,0,1325624400,1325883600,56.90,53.0,3.9,3.90,3.0,0.00000,0.00152,3.85,3.92,22.0,0.0,0.01,587.0,5,s3://tensorflow/renforcement/rawdata/2012-01/Q...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1688,1325883600,1325883600,57.78,60.0,2.2,-2.22,0.0,0.00049,0.00358,0.00,0.01,0.0,2.2,2.29,32.0,12,s3://tensorflow/renforcement/rawdata/2012-01/Q...
60,1688,1325883600,1325883600,57.78,61.0,3.2,-3.22,0.0,0.00054,0.00299,0.00,0.01,0.0,3.2,3.29,0.0,13,s3://tensorflow/renforcement/rawdata/2012-01/Q...
61,1688,1325883600,1325883600,0.00,0.0,0.0,0.00,0.0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00,0.0,14,s3://tensorflow/renforcement/rawdata/2012-01/Q...
62,1688,1325883600,1325883600,0.00,0.0,0.0,0.00,0.0,0.00000,0.00000,0.00,0.00,0.0,0.0,0.00,0.0,15,s3://tensorflow/renforcement/rawdata/2012-01/Q...


In [ ]:
=========================================== TEST ==============================================

In [ ]:
import os
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
print(PartitionDate)

In [ ]:
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        for k in [*f.keys()]:
            DATA = f[k][:]

In [ ]:
df = pd.DataFrame(DATA[0])
df.columns = OTHER_COL+SCALER_COL
df 

In [ ]:
df = pd.DataFrame(DATA[1])
df.columns = OTHER_COL+SCALER_COL
df 

In [ ]:
data_shape

In [ ]:
df = pd.DataFrame(DATA[0])
df.columns = OTHER_COL+SCALER_COL
df

In [ ]:
df = pd.DataFrame(DATA[1])
df.columns = OTHER_COL+SCALER_COL
df

In [ ]:
df = pd.DataFrame(DATA[2])
df.columns = OTHER_COL+SCALER_COL
df

In [ ]:
df = pd.DataFrame(DATA[3])
df.columns = SCALER_COL
df